In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets.mnist import MNIST
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
data_train = MNIST('./data/mnist',
                   download=True,
                   transform=transforms.Compose([
                       transforms.Resize((32, 32)),
                       transforms.ToTensor()]))
data_test = MNIST('./data/mnist',
                  train=False,
                  download=True,
                  transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor()]))

In [3]:
data_train_loader = DataLoader(data_train, batch_size=256, shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test, batch_size=1024, num_workers=8)

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.cnn_l1 = nn.Conv2d(1, 6, kernel_size=(5,5))
        self.cnn_l2 = nn.Conv2d(6, 16, kernel_size=(5,5))
        self.cnn_l3 = nn.Conv2d(16, 120, kernel_size=(5,5))
        
        self.fc_l1 = nn.Linear(48000, 84)
        self.fc_l2 = nn.Linear(84, 10)
    
    def forward(self, img):
        net = F.relu(self.cnn_l1(img))
        net = F.relu(self.cnn_l2(net))
        net = F.relu(self.cnn_l3(net))
 
        net = net.view(-1, 48000)
        net = F.relu(self.fc_l1(net))
        net = F.log_softmax(self.fc_l2(net), dim=1)
        
        return net

In [5]:
network = Network()
network

Network(
  (cnn_l1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (cnn_l2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (cnn_l3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc_l1): Linear(in_features=48000, out_features=84, bias=True)
  (fc_l2): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

In [7]:
for epoch in range(3):
    for i, (images, labels) in tqdm(enumerate(data_train_loader)):
        network.zero_grad()
        
        output = network(images)
        
        loss = F.nll_loss(output, labels) 
        
        loss.backward() 
        optimizer.step()
        
    print(loss) 

235it [02:32,  1.54it/s]

tensor(0.0612, grad_fn=<NllLossBackward>)



235it [02:31,  1.55it/s]

tensor(0.0430, grad_fn=<NllLossBackward>)



235it [02:35,  1.51it/s]

tensor(0.0555, grad_fn=<NllLossBackward>)


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for i, (images, labels) in tqdm(enumerate(data_test_loader)):
        for j in range(len(images)):
            real_class = torch.argmax(labels[j])
            net_out = network(images[j].view(-1, 1, 32, 32))
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1
print("Accuracy: ", correct/total*100)

10it [00:25,  2.53s/it]

Accuracy:  0.098
